In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

First, we load in our dataset

In [4]:
df = pd. read_csv('/Users/jessiefreelander/Flatiron_labs/Git_lab_projects/Phase_1_proj/Phase-1-Project/Phase-1-Project/Phase-1-Project/Phase-1-Project/Phase-1-Project/Phase-1-Project/Data/AviationData.csv', encoding = 'latin1', low_memory = False)
df

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


Let's clean up any unnecessary punctuation, spaces, and fix capitalization for column names.

In [ ]:
#Replace periods in all the column names
df.columns = df.columns.str.replace('.', ' ', regex=False)

In [ ]:
#Fixing the capitalization for column names
df.columns = df.columns.str.title()

In [ ]:
df.info()

Let's start exploring each column to identify any duplicate values, missing values, and unnecessary columns/rows.

In [ ]:
df['Weather Condition'].value_counts()

In [ ]:
#There are two columns for unknowns. Let's consolidate into one.
df['Weather Condition'] = df['Weather Condition'].str.replace('Unk','UNK')

In [ ]:
df['Weather Condition'].value_counts()

In [ ]:
# The publication date for these reports are not useful to us. The event date 
## is enough for us to track the times of these accidents.
df = df.drop('Publication Date', axis = 1)

In [ ]:
df.info()

In [ ]:
df['Schedule'].value_counts()

In [ ]:
# Since the schedule status for flights does not contribute anyway to safety
## of flights we can drop this column.

df = df.drop('Schedule', axis = 1)

In [ ]:
#We will be focusing on the make and model of each plane so we can do without
#   knowing the Air Carriers.

df = df.drop('Air Carrier', axis = 1)

In [ ]:
# There's more missing values than existing values for the Latitude and Longitude columns
## Since we have the cities, countries, and airports to pinpoint locations of flights,
### let's get ride of these 2 columns.

df['Latitude'].isna().value_counts()
df['Longitude'].isna().value_counts()

In [ ]:
df = df.drop('Latitude', axis=1)
df = df.drop('Longitude', axis=1)

In [ ]:
# Amateur Built aircrafts are homebuilt aircrafts or kit planes constructed by
## individuals for personal use or education rather than professional activity.
### We only want professionaly built aircrafts for our purposes, so we can get rid of any
#### aircrafts that are Amateur Built.

df = df.drop(df[df['Amateur Built'] == 'Yes'].index)

In [ ]:
df.info()

In [ ]:
df['Engine Type'].value_counts()

Let's take a deep dive into these engine types to figure out which engines are suitable for our corporate and business aviation purposes.

1. Reciprocating (a.k.a. piston) engines are used for general aviation because they're affordable, robust, and have good fuel efficiency. They're also used for corporate aviation and are best for flying at lower altitudes. **We'll keep this one!**

2. Turbo Shaft engines are used in helicopters and large commercial aircrafts. **We'll keep this!**

3. Turbo Prop engines are a hybrid of piston and jet ingines that are used in light aircrafts. They're known for their fuel efficiency on short and medium-haul flights. They offer good performance at low altitudes and speeds, and are ideal fo take-off and landing operations at smaller airports. **We'll keep this!**

4. Turbofan engines are commonly used in corporate aircrafts including light to mid-size business jets, and are generally popular for commercial aircrafts. They are known for being compact and fuel efficient. **We'll keep this!**

5. Turbo Jet engines are commonly used in commercial aircrafts and private jets. They are capable of high speeds and take up little space. **We'll keep this!**

6. Geared Turbo Fans are commonly used in commercial aircrafts, a sub-type of Turbofan engines. They are efficient with smaller and faster tubrines. **We'll keep this!**

7. LR engines, or long range engines, are developed for ultra-long-haul routes such as from L.A. to Singapore. This will be useful for our executives. **We'll keep this!**

8. Electric engines are relatively newer to the industry and are mainly used for training flights. They're not yet a common use for the commercial industry though many companies are working on and investing in building these aircrafts. We can drop these values and luckily there's only 10 values to drop. 

9. Hybrid rocket engines are used in rockets, so for obvious reasons we can drop that.

In [ ]:
df.drop(df[df['Engine Type'] == 'Hybrid Rocket'].index, inplace = True)
df.drop(df[df['Engine Type'] == 'Electric'].index, inplace = True)

df['Engine Type'].value_counts()

Now let's consolidate the unknown values.

In [ ]:
df['Engine Type'] = df['Engine Type'].str.replace('UNK', 'Unknown')
df['Engine Type'] = df['Engine Type'].str.replace('NONE', 'Unknown')
df['Engine Type'] = df['Engine Type'].str.replace('None', 'Unknown')
df['Engine Type'].value_counts()

In [ ]:
# Let's replace the missing values as Unknown.
df['Engine Type'].isna().value_counts()


In [ ]:
df['Engine Type'] = df['Engine Type'].fillna('Unknown')
df['Engine Type'].value_counts()

In [ ]:
df.info()

In [ ]:
# We won't need the airport codes since we have airport names.
df.drop('Airport Code', axis=1, inplace = True)

In [ ]:
# Let's replace missing Location value as Unknown.
df['Location'].isna().value_counts()
df['Location'] = df['Location'].fillna('Unknown')

In [ ]:
df.info()

In [ ]:
# Let's replace any missing Country values with Unknown
df['Country'] = df['Country'].fillna('Unknown')
df['Country'].value_counts()

In [ ]:
# It looks like Country column has a skewed representation of data because the U.S. represents more than 90% of the data. 
## We can make a separate dataframe that solely includes U.S. called df_us to do analyses on specific incidents in the U.S.

df_us = df[df['Country'] == 'United States']

In [ ]:
# Event Date values are currently an object type, let's transform them into datetime types
df['Event Date'] = pd.to_datetime(df['Event Date'])

In [ ]:
# Injury Severity has Fatal values with different names. Let's rename any values that 
# start with Fatal to just be called 'Fatal'.

df['Injury Severity'] = df['Injury Severity'].apply(lambda x: 'Fatal' if 'Fatal' in str(x) else x)

In [ ]:
# Injury Severity has missing values with different names, let's call them all 'Unavailable'.
df['Injury Severity'].fillna('Unavailable', inplace=True)
df['Injury Severity'].value_counts()


In [ ]:
# There are many types of aircrafts but for our intents and purposes we only want to see airplanes, so let's remove anything that isn't an airplane.
df['Aircraft Category'].value_counts()


In [ ]:
df = df.drop(df[(df['Aircraft Category'] != 'Airplane')].index)

In [ ]:
# Let's look at our overall info ans see what else is left.
df.info()

In [ ]:
# Aircraft Damage has  138 missing values, lets drop them since there's too few to affect our analysis.

df['Aircraft Damage'].value_counts()
df['Aircraft Damage'].isna().value_counts()
df['Aircraft Damage'] = df['Aircraft Damage'].fillna('Unknown')
df = df.drop(df[df['Aircraft Damage'] == 'Unknown'].index)
df['Aircraft Damage'].value_counts()


In [ ]:
# Let's look into the Purpose of Flight column.
## Let's get rid of the values that won't be useful to us for business and corporate travel
df['Purpose Of Flight'].value_counts()


In [ ]:
df = df.drop(df[(df['Purpose Of Flight'] != 'Personal') & (df['Purpose Of Flight'] != 'Aerial Application') & (df['Purpose Of Flight'] != 'Instructional') & (df['Purpose Of Flight'] != 'Unknown') & (df['Purpose Of Flight'] != 'Business') & (df['Purpose Of Flight'] != 'Positioning') & (df['Purpose Of Flight'] != 'Executive/corporate') & (df['Purpose Of Flight'] != 'Flight Test') & (df['Purpose Of Flight'] != 'Other Work Use') & (df['Purpose Of Flight'] != 'Public Aircraft - State') & (df['Purpose Of Flight'] != 'Ferry') & (df['Purpose Of Flight'] != 'Public Aircraft') & (df['Purpose Of Flight'] != 'Public Aircraft - Local')].index)

In [ ]:
# We'll get rid of registration number since that won't be useful to our analysis
df = df.drop(['Registration Number'], axis =1)

In [ ]:
# Models has some missing values that we'll drop since there aren't many and have unknown Models will not be helpful in offering a final recommendation.
df['Model'].isna().value_counts()
df.dropna(subset= ['Model'], inplace=True)

In [ ]:
df['Model'].isna().value_counts()

In [ ]:
# We need to clean up the Make column to get rid of duplicate Make types caused by different cases and white space.
df['Make'].value_counts()
df['Make'] = df['Make'].replace('CESSNA', 'Cessna')
df['Make'] = df['Make'].replace('PIPER', 'Piper')
df['Make'] = df['Make'].replace('BEECH', 'Beech')
df['Make'] = df['Make'].str.strip(' ')
df[['Make', 'Model']].value_counts()
df['Make'] = df['Make'].str.title()
df['Make'].value_counts()

#Let's also drop any missing values in this column
df.dropna(subset= ['Make'], inplace=True)



In [ ]:
df['Make'].isna().value_counts()


In [ ]:
df['Number Of Engines'].isna().value_counts()

In [ ]:
# I wanted to see what the relationship between Total Fatal Injuries and Make looks like.
## This scatter plot shows that there's almost too many Makes to be able to distinguish any type of pattern
### It seems it will be important to cut down on unneccessary Makes and Models, and then create a Make-Model column
#### to have unique identifiers that will stand out and reveal more to us

fig, ax = plt.subplots(figsize = (20, 10))
sns.scatterplot(x = 'Make', y = 'Total Fatal Injuries', data = df, ax = ax)

Let's keep getting rid of missing values in the rest of the rest of the columns first before we continue cutting down Makes and Models.

In [ ]:
# The mean, median, and mode for number of engines is about 1 per aircraft. Let's fill the missing values with the average.
df['Number Of Engines'].mean()
df['Number Of Engines'] = df['Number Of Engines'].fillna(df['Number Of Engines'].mean())
df['Number Of Engines'].isna().value_counts()

In [ ]:
# The mean of total fatal injuries is .04. Theres about 1000 missing values. Let's fill them in with the mean.
df['Total Fatal Injuries'].isna().value_counts()
df['Total Fatal Injuries'].mean()

In [ ]:
df['Total Fatal Injuries'] = df['Total Fatal Injuries'].fillna(df['Total Fatal Injuries'].mean())

In [ ]:
# Theres some missing values here, lets forward fill to fill in the gaps.
df['Total Serious Injuries'].ffill(axis=0, inplace=True)
df['Total Serious Injuries'].isna().value_counts()

In [ ]:
df['Total Minor Injuries'].ffill(axis=0, inplace=True)
df['Total Minor Injuries'].isna().value_counts()

In [ ]:
# Let's fill in these missing values the same way
df['Total Uninjured'].ffill(axis=0,inplace=True)
df['Total Uninjured'].isna().value_counts()

In [ ]:
# We can calculate the number of Total Passengers in each incident by adding up the fatal injuries, serious injuries, minor injuries, and uninjured.
## This will be helpful when we want to build our risk index by calculating the ratio of fatalities and injuries for each make and model.
df['Total Passengers'] = df['Total Fatal Injuries'] + df['Total Minor Injuries'] + df['Total Serious Injuries'] + df['Total Uninjured']

df.info()

In [ ]:
# I will also drop the Report Status since this information will not be helping us calculate the risk index  to measure safety or efficiency of engines
df.drop(['Report Status'], axis = 1, inplace=True)


In [ ]:
# We will keep the Broad Phase of Flight column to see if there is specific when these accidents may occur, or not.
df['Broad Phase Of Flight'] = df['Broad Phase Of Flight'].fillna('Unknown')
df.info()

In [ ]:
# Weather conditions will be interesitng to evaluate how they affect fatalities/injuries and how certain aircrafts perform in worse conditions
df['Weather Condition'].isna().value_counts()

In [ ]:
# Let's fill in the missing values here
df['Weather Condition'].fillna('Unknown', axis=0, inplace=True)

In [ ]:
# We'll do the same for airport Names.
df['Airport Name'] = df['Airport Name'].fillna('Unknown')

In [ ]:
df.dropna(subset=['Amateur Built'], axis=0, inplace=True)

In [ ]:
df['Amateur Built'].isna().value_counts()

We will drop any values with Total Passengers greater than 20 for our business purposes.

In [ ]:
df =  df.drop(df[df['Total Passengers'] > 10].index)
(df['Total Passengers'] > 10).value_counts()


In [ ]:
# Let's remove aircrafts with instructional and aerial application purpose of flight
## these aircrafts do not apply to our 
df_filtered = df[(df['Purpose Of Flight'] != 'Instructional') & (df['Purpose Of Flight'] != 'Public Aircraft') & (df['Purpose Of Flight'] != 'Aerial Application')]
df = df_filtered

In [ ]:
# We can also get rid of the Personal flights because they seem to be unfit for corporate use
df['Purpose Of Flight'].value_counts()
df = df[df['Purpose Of Flight'] != 'Personal']

In [ ]:
# Let's drop all of the makes and models that don't apply for our business purposes
df =  df.drop(df[(df['Make'] == 'Cessna') & (df['Model'] == '172')].index)
df =  df.drop(df[(df['Make'] == 'Cessna') & (df['Model'] == '152')].index)
df =  df.drop(df[(df['Make'] == 'Cessna') & (df['Model'] == '150')].index)
df =  df.drop(df[(df['Make'] == 'Cessna') & (df['Model'] == '172S')].index)
df =  df.drop(df[(df['Make'] == 'Cessna') & (df['Model'] == '172N')].index)


In [ ]:
# Let's drop the make that only have one value counts
make_counts= df['Make'].value_counts()
df_filtered = df[df['Make'].isin(make_counts[make_counts > 1].index)]
df = df_filtered
df['Make'].value_counts() == 1
df['Make'].value_counts()

In [ ]:
# The list below is a list of makes and models put together by CHatGPT
# that are not suited for our business purposes, so we will drop them
remove_list = [
    ("Cessna", "140"),
    ("Bellanca", "17-30A"),
    ("Cessna", "R172K"),
    ("Navion", "A"),
    ("Beech", "19"),
    ("Cessna", "180"),
    ("Cessna", "150L"),
    ("Piper", "PA-24-180"),
    ("Grumman", "AA-5B"),
    ("Beechcraft", "BE-58"),
    ("Maule", "M-5-235C"),
    ("Cessna", "210L"),
    ("Bellanca", "7GCBC"),
    ("Cessna", "182"),
    ("Air Tractor", "AT-301"),
    ("Piper", "PA-28-161"),
    ("Piper", "PA-34-200T"),
    ("Cessna", "T337G"),
    ("Piper", "PA-28R-200"),
    ("Piper", "PA-22")
]

remove_df = pd.DataFrame(remove_list, columns=['Make', 'Model'])
df_filtered = df[~df[['Make', 'Model']].apply(tuple, axis=1).isin(remove_list)]
df = df_filtered


In [ ]:
# In order to make it easier to filter out the makes and models we don't need, 
# we need to combine them into one column.
df[['Make', 'Model']]
df['Make-Model'] = df['Make'] + ' ' + df['Model']
df['Make-Model']


In [ ]:
make_model_counts= df['Make-Model'].value_counts()
df2 = df[df['Make-Model'].isin(make_model_counts[make_model_counts > 1].index)]
df = df2

In [ ]:
df['Make-Model'].value_counts()

In [ ]:
df = df.drop(columns = ['Accident Number', 'Far Description', 'Event Id'], axis=1)

In [ ]:
df.set_index('Event Date')

In [ ]:
df = df.drop(['Aircraft Category'], axis =1)

In [3]:
df.to_csv('Acessable_Cleaned_Data_Aviation')

NameError: name 'df' is not defined